In [1]:
import pandas as pd


In [2]:
#1. Read the path of each document
ai_path = r"D:\25Spring\MSE641\research\ai_model_predictions.csv\ai_model_predictions.csv"
gpatent_path = r"D:\patent research\g_patent.tsv\g_patent.tsv"
gassignee_path = r"D:\patent research\g_assignee_disambiguated.tsv\g_assignee_disambiguated.tsv"
pvabstract_path = r"E:\Waterloo-onedrive\OneDrive - University of Waterloo\MSE 641\g_patent_abstract.tsv\g_patent_abstract.tsv"

In [3]:
gpatent_usecols = [
    'patent_id', 'patent_type', 'patent_date',
    'patent_title', 'num_claims', 'withdrawn'
]
gassignee_usecols = [
    'patent_id', 'disambig_assignee_organization', 'assignee_type'
]
gpatent_df = pd.read_csv(
    gpatent_path,
    sep='\t',
    usecols=gpatent_usecols,
    dtype=str,
    low_memory=True
)
gassignee_df = pd.read_csv(
    gassignee_path,
    sep='\t',
    usecols=gassignee_usecols,
    dtype=str,
    low_memory=True
)
pv_abstract = pd.read_csv(pvabstract_path, sep = '\t')
#2. Merge the data based on the unique id
# normalize the data
for df in [gpatent_df, gassignee_df,pv_abstract]:
    df['patent_id'] = df['patent_id'].astype(str) \
                                        .str.strip() \
                                        .str.upper()
gpatent_df = gpatent_df[gpatent_df['withdrawn'] != '1'].drop(columns=['withdrawn'])
pv_merged = pd.merge(
    gpatent_df,
    gassignee_df,
    on='patent_id',
    how='inner',
    indicator=True
)

# print the result
print(pv_merged['_merge'].value_counts())
pv_merged = pv_merged.drop(columns=['_merge'])



C:\Users\Yue Qiao\AppData\Local\Temp\ipykernel_13908\1595735438.py:22: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  pv_abstract = pd.read_csv(pvabstract_path, sep = '\t')


_merge
both          8451107
left_only           0
right_only          0
Name: count, dtype: int64


In [4]:
#3. Drop unwanted columns and duplicate patent_id
pv_merged = pv_merged.drop_duplicates(subset=['patent_id'])
#5. Convert patent_date to datetime format (MM/DD/YYYY)

#6. Drop off unnecessary columns and data
pv_merged['patent_date'] = pd.to_datetime(
    pv_merged['patent_date'],
    format='%Y-%m-%d',
    errors='coerce'
)
filtered_df = pv_merged[pv_merged['patent_date'].dt.year >= 2015]

pv_final = pd.merge(
    filtered_df,
    pv_abstract,
    on='patent_id',
    how='inner',
    indicator=True
)

print(pv_final['_merge'].value_counts())

_merge
both          3401581
left_only           0
right_only          0
Name: count, dtype: int64


In [5]:
# ai_columns = pd.read_csv(ai_path, sep='\t', dtype=str, nrows=0).columns.tolist()
# print(ai_columns)

In [6]:

ai_df = pd.read_csv(ai_path, dtype=str, low_memory=True)
cols_to_drop = ['flag_patent', 'appl_id', 'predict50_any_ai', 'predict86_any_ai']
ai_df = ai_df.drop(columns=cols_to_drop)
ai_df = ai_df[ai_df['predict93_any_ai'] == '1'].copy()
ai_df['pub_dt'] = pd.to_datetime(ai_df['pub_dt'], errors='coerce')
ai_df = ai_df[ai_df['pub_dt'].dt.year >= 2015]
ai_df['doc_id_clean'] = ai_df['doc_id'].str.strip().str.upper()




In [10]:
pv_final.drop(columns=['_merge'], errors='ignore', inplace=True)
merged_df = pd.merge(
    ai_df,
    pv_final,
    left_on='doc_id_clean',
    right_on='patent_id',
    how='inner',
    indicator=True
)
print(merged_df['_merge'].value_counts())


_merge
both          528673
left_only          0
right_only         0
Name: count, dtype: int64


In [ ]:
cols_to_drop = [
    'predict93_any_ai',
    'doc_id_clean',
    'doc_id',
    'patent_date',
    '_merge'
]
prefix_cols_to_drop = [col for col in merged_df.columns if col.startswith('predict50_') or col.startswith('predict86')]
all_cols_to_drop = cols_to_drop + prefix_cols_to_drop
merged_df = merged_df.drop(columns=all_cols_to_drop)
output_path = r"E:\Waterloo-onedrive\OneDrive - University of Waterloo\MSE 641\Merged Data\Merged_data_original.csv"
merged_df.to_csv(output_path, index=False)

In [4]:
# Confirm the final merge data

import pandas as pd
input_path = r"E:\Waterloo-onedrive\OneDrive - University of Waterloo\MSE 641\Merged Data\Merged_data_original.csv"
patent_df = pd.read_csv(input_path, sep=',', encoding='utf-8')
ml_df = patent_df[patent_df['predict93_ml'] == 1]
ml_df['patent_text'] = (ml_df['patent_title'].fillna('') + '. ' + ml_df['patent_abstract'].fillna('')).str.strip()

output_path = r"E:\Waterloo-onedrive\OneDrive - University of Waterloo\MSE 641\Merged Data\Merged_data_ML.csv"
ml_df.to_csv(output_path, index=False)

C:\Users\Yue Qiao\AppData\Local\Temp\ipykernel_34940\625613034.py:5: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  patent_df = pd.read_csv(input_path, sep=',', encoding='utf-8')
C:\Users\Yue Qiao\AppData\Local\Temp\ipykernel_34940\625613034.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ml_df['patent_text'] = (ml_df['patent_title'].fillna('') + '. ' + ml_df['patent_abstract'].fillna('')).str.strip()
